In [1]:
'''
載入tensroflow模組
先搜尋本地資料夾
若沒有則到網路上載
'''

def set_model(model_name):
#     model_found = 0
#     for file in glob.glob('*'):     
#         if (file == model_name):
#             model_found = 1
    base = 'models/'
    model_name = base + model_name
    model_file = model_name + '.tar.gz'
    download_base = 'http://download.tensorflow.org/models/object_detection/'
    path_to_ckpt = model_name + '/frozen_inference_graph.pb'
    path_to_labels = os.path.join('data', 'mscoco_label_map.pbtxt')
    num_classes = 90
#     if (model_found == 0):		
#         opener = urllib.request.URLopener()
#         opener.retrieve(download_base + model_file, model_file)
#         tar_file = tarfile.open(model_file)
#         for file in tar_file.getmembers():
#             file_name = os.path.basename(file.name)
#             if 'frozen_inference_graph.pb' in file_name:
#                 tar_file.extract(file, os.getcwd())
    detection_graph = tf.Graph()
    with detection_graph.as_default():
        od_graph_def = tf.GraphDef()
        with tf.gfile.GFile(path_to_ckpt, 'rb') as fid:
            serialized_graph = fid.read()
            od_graph_def.ParseFromString(serialized_graph)
            tf.import_graph_def(od_graph_def, name='')
    label_map = label_map_util.load_labelmap(path_to_labels)
    categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=num_classes, use_display_name=True)
    category_index = label_map_util.create_category_index(categories)
    
    return detection_graph, category_index
'''
回傳detection_graph =>tensorflow的模組
category_index=> 結果的分類表   ex class:1 == 人, class:2 == 腳踏車.....
'''

'\n回傳detection_graph =>tensorflow的模組\ncategory_index=> 結果的分類表   ex class:1 == 人, class:2 == 腳踏車.....\n'

In [2]:
def dected_ROI(frame, width, height ,limit):   #保留frame [limit:height-limit, limit: width-limit] 範圍內的值  其餘弄成黑色
#     margin = limit//2
    mask=np.zeros([height,width],dtype=np.uint8) 
    mask[limit:height-limit, limit: width-limit] = 255
    image = cv2.bitwise_and(frame, frame, mask=mask) 
    return image

In [3]:
def get_background(frame, vname):
    path = 'save/background_{}.jpg'.format(vname)
    cv2.imwrite(path, frame)

In [4]:

def main_func_predict(detection_graph, category_index, video_path, 
                    dected_per_frame = 30, method = 'dlib', 
                      seconds_proid = 5):
    getFrame = False
    video_name = re.findall(r'/(\w+)\.mp4$', video_path)[0]
    fm = filed_tool_v2.All_filed(video_name, seconds_proid)

    param = {'fm' : fm} 

    def draw_rectangle(event,x,y,flags,param):  #setMouseCallback要用的
        if event == cv2.EVENT_LBUTTONDOWN:
            param['fm'].updateI((x,y))
        elif event == cv2.EVENT_LBUTTONUP:
            param['fm'].updateFiled((x,y))
#     param['fm'].testdata()


    cv2.namedWindow(winname='my_drawing')
    cv2.setMouseCallback('my_drawing',draw_rectangle,param)
    
    
    cap = cv2.VideoCapture(video_path)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    limit = int(max(width, height)/10)
#     width = 1000
#     height = 1000
    dis_threshold = max(height, width)/3
    fps = cap.get(cv2.CAP_PROP_FPS)
    
    write_path = 'result/{}.mp4'.format(video_name)
    videoWriter = cv2.VideoWriter(write_path, cv2.VideoWriter_fourcc(*'DIVX'), fps, (width, height))
    total_frame = 0
    sec = 0
    ft = Filter.Filter(width, height) 
    dt = draw_tool_v2.draw_tool()
    print("width:{}".format(width))
    print("height:{}".format(height))
    trks = mutiTracker.Trackers(dist_thresh = 300, max_frames_to_skip = 4, 
                                max_trace_length = 600, 
                                leave_limit = limit/2, 
                                frame_info = (width, height),
                               video_name = video_name,
                               time_proid = seconds_proid)
    peroid_count = False
    with detection_graph.as_default():
         with tf.Session(graph=detection_graph) as sess:
            image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
            detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
            detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
            detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')
            num_detections = detection_graph.get_tensor_by_name('num_detections:0')

            while(cap.isOpened()):
                ret,frame = cap.read()
                if not ret:
                    print("end of the video file...")
                    break
                if method == 'dlib':
                    dected_frame = frame
                    dected_frame = cv2.cvtColor(dected_frame, cv2.COLOR_BGR2RGB)
                else:
                    dected_frame = frame
                
                if len(trks.get_trackers()) != 0:
                    trks.predict_update(dected_frame)
                    trackers = trks.get_trackers()
                    for tracker in trackers:
                        cen = tracker.centroid
                        ID = tracker.track_id
                        box = tracker.bbox
                        cv2.rectangle(frame, (box[0], box[1]), (box[2], box[3]),(0, 255, 0), 1)
                        cv2.putText(frame, str(ID), (cen[0], cen[1]), cv2.FONT_HERSHEY_SIMPLEX,0.5, (0, 255, 255), 1, cv2.LINE_AA)
                    trks.checked_leaved()
                    
                if total_frame % dected_per_frame == 0:
                    print("dected!!!!!!!!!")
                    tensor_frame = dected_ROI(frame, width, height ,limit)
                    image_np_expanded = np.expand_dims(tensor_frame, axis=0)
                    (boxes, scores, classes, num) = sess.run(
                        [detection_boxes, detection_scores, detection_classes, num_detections],
                        feed_dict={image_tensor: image_np_expanded})

                    rects, total_count, adds = ft.predict_Filter_people_box(np.squeeze(boxes), 
                                                                            np.squeeze(classes).astype(np.int32),
                                                                            np.squeeze(scores))
             
                    if not getFrame:
                        get_background(frame, video_name)
                        getFrame = True
                    for (startX, startY, endX, endY) in rects:
                        cv2.rectangle(frame, (startX, startY), (endX, endY),(0, 0, 255), 3)

                    trks.detect_Update(rects, dected_frame, method)
                

                fm.dected_per_frame(trks.get_trackers(), frame)
                if fm.is_has_filed():
                    dt.draw_filed(frame, fm.get_all_filed())
                videoWriter.write(frame)
                cv2.imshow('my_drawing', frame)
                total_frame+=1
                
                sec = int(round(total_frame/fps,3))
#                 print("sec{}".format(sec))
                
                if sec % seconds_proid == 0 and int(sec)!=0 and not peroid_count:
                    time_proid = "{}_{}".format(int(sec-seconds_proid), int(sec))
                    trks.time_proid = time_proid
                    fm.time_proid = time_proid
                    trks.save_total_people()
                    fm.save_all_count()
                    peroid_count = True
                elif sec % seconds_proid != 0:
                    peroid_count = False
                print('-'*50)
                if cv2.waitKey(25) & 0xFF == ord('q'):
                    break
                    
    fm.final_save_all()
    cap.release()
    cv2.destroyAllWindows()


In [5]:
import numpy as np
import time, re, imutils, glob, os, cv2
import tensorflow as tf
import numpy as np
from utils_v2 import filed_tool_v2, label_map_util, Filter, draw_tool_v2
from trackers import mutiTracker, correlation_tracker, kalman_filter, opencv_trackers

detection_graph, category_index =set_model('faster_rcnn_resnet101_coco_11_06_2017')
main_func_predict(detection_graph,category_index, 'videos/v51.mp4')

width:1920
height:1080
dected!!!!!!!!!
--------------------------------------------------
 in checked leave
--------------------------------------------------
 in checked leave
--------------------------------------------------
 in checked leave
--------------------------------------------------
 in checked leave
--------------------------------------------------
 in checked leave
--------------------------------------------------
 in checked leave
--------------------------------------------------
 in checked leave
--------------------------------------------------
 in checked leave
--------------------------------------------------
 in checked leave
--------------------------------------------------
 in checked leave
--------------------------------------------------
 in checked leave
--------------------------------------------------
 in checked leave
--------------------------------------------------
 in checked leave
--------------------------------------------------
 in checked l

 in checked leave
filed id:1 dected_people dict id: 0, value:{'stay_time': 0.7, 'is_still_in': 1, 'is_counted': False, 'stay_frame': 21}
filed id:1 dected_people dict id: 1, value:{'stay_time': 0.7, 'is_still_in': 1, 'is_counted': False, 'stay_frame': 21}
--------------------------------------------------
 in checked leave
filed id:1 dected_people dict id: 0, value:{'stay_time': 0.733, 'is_still_in': 1, 'is_counted': False, 'stay_frame': 22}
filed id:1 dected_people dict id: 1, value:{'stay_time': 0.733, 'is_still_in': 1, 'is_counted': False, 'stay_frame': 22}
--------------------------------------------------
 in checked leave
filed id:1 dected_people dict id: 0, value:{'stay_time': 0.767, 'is_still_in': 1, 'is_counted': False, 'stay_frame': 23}
filed id:1 dected_people dict id: 1, value:{'stay_time': 0.767, 'is_still_in': 1, 'is_counted': False, 'stay_frame': 23}
--------------------------------------------------
 in checked leave
filed id:1 dected_people dict id: 0, value:{'stay_tim

 in checked leave
filed id:1 dected_people dict id: 0, value:{'stay_time': 1.4, 'is_still_in': 1, 'is_counted': False, 'stay_frame': 42}
filed id:1 dected_people dict id: 1, value:{'stay_time': 1.4, 'is_still_in': 1, 'is_counted': False, 'stay_frame': 42}
filed id:2 dected_people dict id: 2, value:{'stay_time': 0.4, 'is_still_in': 1, 'is_counted': False, 'stay_frame': 12}
--------------------------------------------------
 in checked leave
filed id:1 dected_people dict id: 0, value:{'stay_time': 1.433, 'is_still_in': 1, 'is_counted': False, 'stay_frame': 43}
filed id:1 dected_people dict id: 1, value:{'stay_time': 1.433, 'is_still_in': 1, 'is_counted': False, 'stay_frame': 43}
filed id:2 dected_people dict id: 2, value:{'stay_time': 0.433, 'is_still_in': 1, 'is_counted': False, 'stay_frame': 13}
--------------------------------------------------
 in checked leave
filed id:1 dected_people dict id: 0, value:{'stay_time': 1.467, 'is_still_in': 1, 'is_counted': False, 'stay_frame': 44}
fil

 in checked leave
filed id:1 dected_people dict id: 0, value:{'stay_time': 2.1, 'is_still_in': 1, 'is_counted': False, 'stay_frame': 63}
filed id:1 dected_people dict id: 1, value:{'stay_time': 2.1, 'is_still_in': 1, 'is_counted': False, 'stay_frame': 63}
filed id:2 dected_people dict id: 2, value:{'stay_time': 1.1, 'is_still_in': 1, 'is_counted': False, 'stay_frame': 33}
--------------------------------------------------
 in checked leave
filed id:1 dected_people dict id: 0, value:{'stay_time': 2.133, 'is_still_in': 1, 'is_counted': False, 'stay_frame': 64}
filed id:1 dected_people dict id: 1, value:{'stay_time': 2.133, 'is_still_in': 1, 'is_counted': False, 'stay_frame': 64}
filed id:2 dected_people dict id: 2, value:{'stay_time': 1.133, 'is_still_in': 1, 'is_counted': False, 'stay_frame': 34}
--------------------------------------------------
 in checked leave
filed id:1 dected_people dict id: 0, value:{'stay_time': 2.167, 'is_still_in': 1, 'is_counted': False, 'stay_frame': 65}
fil

 in checked leave
filed id:1 dected_people dict id: 0, value:{'stay_time': 2.8, 'is_still_in': 1, 'is_counted': False, 'stay_frame': 84}
filed id:1 dected_people dict id: 1, value:{'stay_time': 2.8, 'is_still_in': 1, 'is_counted': False, 'stay_frame': 84}
filed id:2 dected_people dict id: 2, value:{'stay_time': 1.8, 'is_still_in': 1, 'is_counted': False, 'stay_frame': 54}
--------------------------------------------------
 in checked leave
filed id:1 dected_people dict id: 0, value:{'stay_time': 2.833, 'is_still_in': 1, 'is_counted': False, 'stay_frame': 85}
filed id:1 dected_people dict id: 1, value:{'stay_time': 2.833, 'is_still_in': 1, 'is_counted': False, 'stay_frame': 85}
filed id:2 dected_people dict id: 2, value:{'stay_time': 1.833, 'is_still_in': 1, 'is_counted': False, 'stay_frame': 55}
--------------------------------------------------
 in checked leave
filed id:1 dected_people dict id: 0, value:{'stay_time': 2.867, 'is_still_in': 1, 'is_counted': False, 'stay_frame': 86}
fil

...

 in checked leave
filed id:1 dected_people dict id: 11, value:{'stay_time': 4.267, 'is_still_in': 0, 'is_counted': False, 'stay_frame': 128}
filed id:1 dected_people dict id: 15, value:{'stay_time': 4.233, 'is_still_in': 1, 'is_counted': False, 'stay_frame': 127}
filed id:2 dected_people dict id: 16, value:{'stay_time': 4.233, 'is_still_in': 1, 'is_counted': False, 'stay_frame': 127}
filed id:2 dected_people dict id: 11, value:{'stay_time': 20.1, 'is_still_in': 0, 'is_counted': True, 'stay_frame': 603}
--------------------------------------------------
 in checked leave
filed id:1 dected_people dict id: 11, value:{'stay_time': 4.267, 'is_still_in': 0, 'is_counted': False, 'stay_frame': 128}
filed id:1 dected_people dict id: 15, value:{'stay_time': 4.267, 'is_still_in': 1, 'is_counted': False, 'stay_frame': 128}
filed id:2 dected_people dict id: 16, value:{'stay_time': 4.267, 'is_still_in': 1, 'is_counted': False, 'stay_frame': 128}
filed id:2 dected_people dict id: 11, value:{'stay_tim

 in checked leave
filed id:1 dected_people dict id: 11, value:{'stay_time': 4.267, 'is_still_in': 0, 'is_counted': False, 'stay_frame': 128}
filed id:1 dected_people dict id: 15, value:{'stay_time': 4.733, 'is_still_in': 1, 'is_counted': False, 'stay_frame': 142}
filed id:2 dected_people dict id: 16, value:{'stay_time': 4.733, 'is_still_in': 1, 'is_counted': False, 'stay_frame': 142}
filed id:2 dected_people dict id: 11, value:{'stay_time': 20.1, 'is_still_in': 0, 'is_counted': True, 'stay_frame': 603}
--------------------------------------------------
 in checked leave
id:15 about to be delete !!
filed id:1 dected_people dict id: 11, value:{'stay_time': 4.267, 'is_still_in': 0, 'is_counted': False, 'stay_frame': 128}
filed id:2 dected_people dict id: 16, value:{'stay_time': 4.767, 'is_still_in': 1, 'is_counted': False, 'stay_frame': 143}
filed id:2 dected_people dict id: 11, value:{'stay_time': 20.1, 'is_still_in': 0, 'is_counted': True, 'stay_frame': 603}
----------------------------

 in checked leave
filed id:1 dected_people dict id: 11, value:{'stay_time': 4.267, 'is_still_in': 0, 'is_counted': False, 'stay_frame': 128}
filed id:2 dected_people dict id: 16, value:{'stay_time': 5.333, 'is_still_in': 1, 'is_counted': False, 'stay_frame': 160}
filed id:2 dected_people dict id: 11, value:{'stay_time': 20.1, 'is_still_in': 0, 'is_counted': True, 'stay_frame': 603}
--------------------------------------------------
 in checked leave
filed id:1 dected_people dict id: 11, value:{'stay_time': 4.267, 'is_still_in': 0, 'is_counted': False, 'stay_frame': 128}
filed id:2 dected_people dict id: 16, value:{'stay_time': 5.367, 'is_still_in': 1, 'is_counted': False, 'stay_frame': 161}
filed id:2 dected_people dict id: 11, value:{'stay_time': 20.1, 'is_still_in': 0, 'is_counted': True, 'stay_frame': 603}
--------------------------------------------------
 in checked leave
filed id:1 dected_people dict id: 11, value:{'stay_time': 4.267, 'is_still_in': 0, 'is_counted': False, 'stay_

--------------------------------------------------
 in checked leave
filed id:1 dected_people dict id: 11, value:{'stay_time': 4.267, 'is_still_in': 0, 'is_counted': False, 'stay_frame': 128}
filed id:2 dected_people dict id: 16, value:{'stay_time': 5.967, 'is_still_in': 1, 'is_counted': False, 'stay_frame': 179}
filed id:2 dected_people dict id: 11, value:{'stay_time': 20.1, 'is_still_in': 0, 'is_counted': True, 'stay_frame': 603}
--------------------------------------------------
 in checked leave
filed id:1 dected_people dict id: 11, value:{'stay_time': 4.267, 'is_still_in': 0, 'is_counted': False, 'stay_frame': 128}
filed id:2 dected_people dict id: 16, value:{'stay_time': 6.0, 'is_still_in': 1, 'is_counted': False, 'stay_frame': 180}
filed id:2 dected_people dict id: 11, value:{'stay_time': 20.1, 'is_still_in': 0, 'is_counted': True, 'stay_frame': 603}
--------------------------------------------------
 in checked leave
dected!!!!!!!!!
detections:[(1210, 390, 1245, 515), (1201, 38

--------------------------------------------------
 in checked leave
filed id:1 dected_people dict id: 11, value:{'stay_time': 4.267, 'is_still_in': 0, 'is_counted': False, 'stay_frame': 128}
filed id:2 dected_people dict id: 16, value:{'stay_time': 6.567, 'is_still_in': 1, 'is_counted': False, 'stay_frame': 197}
filed id:2 dected_people dict id: 11, value:{'stay_time': 20.667, 'is_still_in': 1, 'is_counted': True, 'stay_frame': 620}
--------------------------------------------------
 in checked leave
filed id:1 dected_people dict id: 11, value:{'stay_time': 4.267, 'is_still_in': 0, 'is_counted': False, 'stay_frame': 128}
filed id:2 dected_people dict id: 16, value:{'stay_time': 6.6, 'is_still_in': 1, 'is_counted': False, 'stay_frame': 198}
filed id:2 dected_people dict id: 11, value:{'stay_time': 20.7, 'is_still_in': 1, 'is_counted': True, 'stay_frame': 621}
--------------------------------------------------
 in checked leave
filed id:1 dected_people dict id: 11, value:{'stay_time': 4

In [6]:

# import numpy as np
# import time, re, imutils, glob, os, cv2
# import tensorflow as tf
# import numpy as np
# from utils_v2 import filed_tool_v2, label_map_util, Filter, draw_tool_v2
# from trackers import mutiTracker, correlation_tracker, kalman_filter, opencv_trackers

# detection_graph, category_index =set_model('faster_rcnn_resnet101_coco_11_06_2017')
# videos = ['v21','v22','v23','v24','v25','v26','v27','v28','v29','v30','v31','v32','v33','v34','v35']
# for video in videos:
#     path = 'videos/{}.mp4'.format(video)
#     main_func_predict(detection_graph,category_index, path)

